In [1]:
import math
import itertools
import collections

def get_input(day):
    filename = 'input/day{}.txt'.format(day)
    with open(filename) as f:
        return [line.strip() for line in f]

# [Day 1: Inverse Captcha](http://adventofcode.com/2017/day/1)

## Part I

**Task:** Given a circular sequence of digits, find the sum of all digits that match the next digit in the list.

**Examples**:
* `1122` -> `3`
* `91212129` -> `9`

In [2]:
def solve_captcha(sequence):
    total = 0
    previous_number = sequence[-1]
    for number in sequence:
        if previous_number == number:
            total += int(previous_number)
        previous_number = number
    return total

day1_input = get_input(1)[0]
solve_captcha(day1_input)

1341

## Part II

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. The list has an even number of elements.

**Example:**

* `1212` -> `6`
* `123123` -> `12`

In [3]:
def solve_halfway_captcha(sequence):
    total = 0
    for index in range(len(sequence) // 2):
        first_num = int(sequence[index])
        second_num = int(sequence[index + len(sequence) // 2])
        if first_num == second_num:
            total += first_num + second_num
    return total    
    
solve_halfway_captcha(day1_input)

1348

# [Day 2 - Corruption Checksum](http://adventofcode.com/2017/day/2)

## Part I

**Task:** Given a spreadsheet of rows and numbers, calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

**Example:**
```
5 1 9 5
7 5 3
2 4 6 8
```
The checksum is `8 + 4 + 6 = 18`.

In [4]:
def calculate_checksum(spreadsheet):
    return sum([max(row) - min(row) for row in spreadsheet])
    
def build_spreadsheet(rows):
    return [list(map(int, row.split('\t'))) for row in rows]
    
spreadsheet = build_spreadsheet(get_input(2))
calculate_checksum(spreadsheet)

50376

## Part II

Now, instead of calculating the difference between the max and min, the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

**Example:**
```
5 9 2 8
9 4 7 3
3 8 6 5
```
The checksum is `4 + 3 + 2 = 9`.

In [5]:
def calculate_division_checksum(spreadsheet):
    total = 0
    for row in spreadsheet:
        total += [b // a for a, b in itertools.combinations(sorted(row), 2) if b % a == 0][0]
    return total

calculate_division_checksum(spreadsheet)

267

# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

## Part I

**Task:** Given an infinite grid allcated in a spiral pattern starting at a location marked `1` and then counting up while spiraling outward. Compute the Manhattan Distance between a given location and square `1`.

**Example:**
```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
* Distance to square `12` is 3 steps.
* Distance to square `23` is 2 steps.
* Distance to square `1024` is 31 steps.

In [6]:
def get_side_length(number):
    nearest_odd_sq = math.ceil(math.sqrt(number))
    nearest_odd_sq = nearest_odd_sq + 1 if nearest_odd_sq % 2 == 0 else nearest_odd_sq
    return nearest_odd_sq

def steps_to_num(side_length, number):
    min_steps_to_num = side_length
    nearest_odd_sq = side_length ** 2
    for i in range(4):
        side_mid_point = nearest_odd_sq - math.floor(side_length / 2) - (side_length - 1) * i
        min_steps_to_num = min(min_steps_to_num, abs(side_mid_point - number))
    return min_steps_to_num
        
def get_steps(number):
    """The level the number is in can be determined by finding the next perfect odd square.
    The perfect odd square is always in the bottom right of each square (e.g. 9, 25, ...).
    The side length is the base of the square. The middle points of a square are the points with the lowest distance from 1.
    E.g. 2, 4, 6, and 8 for the 2nd layer. The distance to these middle points is (side length - 1) / 2.
    Finally, we need to determine the distance from the middle points of a square to the number we are looking for.
    """
    side_length = get_side_length(number)
    steps_to_square = (side_length - 1) / 2
    steps_from_square_to_num = steps_to_num(side_length, number)
    return steps_to_square + steps_from_square_to_num

day3_input = 361527
get_steps(day3_input)

326.0

## Part II

Fill in numbers, starting at `1`, in the same spiral allocation order. Store the sum of all already written values in adjacent squares, including diagonals. Find the first value written that is larger than a given input.

**Example:**
* Square 1 starts with the value 1.
* Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
* Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
* Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

In [7]:
def spiral_generator():
    """Generate spiral coordinates.
    Direction: right, up, left, down, ...
    Step count: 1, 1, 2, 2, 3, 3, 4..."""
    right = lambda pos: (pos[0] + 1, pos[1])
    left = lambda pos: (pos[0] - 1, pos[1])
    up = lambda pos: (pos[0], pos[1] + 1)
    down = lambda pos: (pos[0], pos[1] - 1)
    moves = itertools.cycle([right, up, left, down])
    
    position = (0, 0)
    number_of_moves = 1
    while True:
        for _ in range(2):
            move = next(moves)
            for i in range(number_of_moves):
                yield position
                position = move(position)
        number_of_moves += 1  
    
def get_neighbors(position, values):
    x, y = position
    positions = [(x+1, y), (x-1, y), (x, y+1), (x, y-1),
                 (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1)]
    return [values[p] if p in values else 0 for p in positions]
    
def find_in_sequence(number):
    values = {}
    for position in spiral_generator():
        if position == (0, 0):
            values[position] = 1
            continue
        neighbor_sum = sum(get_neighbors(position, values))
        if neighbor_sum > number:
            return neighbor_sum
        values[position] = neighbor_sum
        
find_in_sequence(day3_input)

363010

# [Day 4: High-Entropy Passphrases](http://adventofcode.com/2017/day/4)

## Part I

Given a passphrase consisting of a series of words (lowercase letters) separated by spaces. A passphrase is valid if it does not contain duplicate words. Given a list of passphrases, return how many passphrases are valid.

**Example:**
* `aa bb cc dd ee` is valid.
* `aa bb cc dd aa` is not valid

In [8]:
passphrases = get_input(4)

def is_unique_passphrase(passphrase):
    counter = collections.Counter(passphrase.split())
    return counter.most_common(1)[0][1] == 1

def count_unique_passphrases(passphrases):
    count = 0
    for passphrase in passphrases:
        if is_unique_passphrase(passphrase):
            count += 1
    return count

count_unique_passphrases(passphrases)

383

## Part II

Now, a valid passphrase must contain no two words that are anagrams of each other.

**Example:**
* `abcde fghij` is valid.
* `abcde xyz ecdab` is not valid.

In [9]:
def is_no_anagram_passphrase(passphrase):
    words = set()
    for word in passphrase.split():
        sorted_word = ''.join(sorted(word))
        if sorted_word in words:
            return False
        words.add(sorted_word)
    return True

def count_no_anagram_passphrases(passphrases):
    count = 0
    for passphrase in passphrases:
        if is_no_anagram_passphrase(passphrase):
            count += 1
    return count

count_no_anagram_passphrases(passphrases)

265

# [Day 5: A Maze of Twisty Trampolines, All Alike](http://adventofcode.com/2017/day/5)

## Part I

Given a list of offsets for each jump, how many times can you jump until you jump out of the list? E.g. an offset of `2` means to two fields forward whereas an offset of `-3` means to move back three fields. After each jump, the offset is increased by `1`.

**Example:**
* List of offsets: `0, 3, 0, 1, -3`. The exist is reached after `5` steps. The offsets after the last iteration are `2, 5, 0, 1, -2`

In [10]:
instructions = list(map(int, get_input(5)))

def count_steps(instructions):
    instructions = list(instructions)
    index = 0
    steps = 0
    while index in range(len(instructions)):
        next_index = index + instructions[index]
        instructions[index] += 1
        index = next_index
        steps += 1
    return steps

count_steps(instructions)

355965

## Part II

Now, after each jump, if the offset was three or more, decrease it by 1. Otherwise, increase it by 1 as before.

In [11]:
def count_steps_with_offset(instructions):
    instructions = list(instructions)
    index = 0
    steps = 0
    while index in range(len(instructions)):
        next_index = index + instructions[index]
        if instructions[index] >= 3:
            instructions[index] -= 1
        else:
            instructions[index] += 1
        index = next_index
        steps += 1
    return steps

count_steps_with_offset(instructions)

26948068